# Multi-Agent Financial Analysis System

This project presents a Multi-Agent Financial Analysis System powered by Agentic AI, designed to emulate the complex, end-to-end analytical workflows used in modern investment firms. Unlike traditional static pipelines, this system leverages a network of autonomous, specialized AI agents that can reason, plan, act, and collaborate dynamically to perform financial analysis with minimal human intervention.  

Agentic AI represents the next evolution in automation—moving beyond linear, rule-based logic to adaptive intelligence that can self-organize, self-critique, and iteratively improve. In this architecture, multiple agents, each with distinct expertise such as data retrieval, financial modeling, sentiment analysis, and investment evaluation—coordinate through a shared reasoning framework. This enables the system to handle real-world financial tasks such as parsing earnings reports, analyzing market news, comparing valuation metrics, and generating investment insights.  

By integrating reasoning, planning, and autonomous coordination, the system mirrors the workflows of professional analysts and research teams. It can:
- Retrieve and process live financial data and news.
- Conduct structured equity and portfolio analyses.
- Evaluate company fundamentals and generate insights.
- Critique and refine its own outputs for improved accuracy.  
- Review the final summary using another grador agent.  

Ultimately, this project demonstrates how Agentic AI architectures can transform traditional financial analysis into a collaborative, intelligent ecosystem that scales analytical reasoning, enhances decision quality, and adapts continuously to new market conditions.

**Git Hub Link** : https://github.com/santausd/Investment-Research-Agent

In [38]:
# ! pip install langchain langchain-core langchain-community

## Import the required libraries

In [39]:
import json
import os
import re
import google.generativeai as genai

import numpy as np
import ollama
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import traceback

import yfinance as yf
from datetime import datetime, timedelta
from newsapi import NewsApiClient
from fredapi import Fred
from sec_api import QueryApi
import requests

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_classic.chains import RetrievalQA
from langchain_text_splitters import RecursiveCharacterTextSplitter # Split texts into manageble chunks
from langchain_classic.embeddings import HuggingFaceEmbeddings
# from langchain_classic.embeddings import HuggingFaceEmbeddings # Converts text chunk into numerical vector
from langchain_classic.vectorstores import FAISS # Helps in finding the similarity between texts in the question and the document
from langchain_core.documents import Document

## Libraries and Tools

This project integrates a combination of **AI, data processing, and financial analysis** libraries to support the multi-agent financial system.

### Core Python & Utilities
- **`json`, `os`, `re`, `traceback`** — Handle data serialization, file operations, regex parsing, and error tracking.  
- **`datetime`, `timedelta`** — Manage date and time calculations for financial data analysis.  
- **`requests`** — Make HTTP requests to external APIs.

### AI and LLM Interfaces
- **`google.generativeai`, `ollama`, `openai`, `ChatGoogleGenerativeAI`** — Provide access to multiple large language models (LLMs) for reasoning, text generation, and multi-agent communication.

### Embeddings and Similarity Search
- **`sentence_transformers`, `HuggingFaceEmbeddings`** — Convert text into numerical vector representations for semantic understanding.  
- **`sklearn.metrics.pairwise.cosine_similarity`** — Measures similarity between vectorized texts.  
- **`FAISS`** — Efficient vector store for document retrieval and similarity search.

### LangChain Components
- **`RetrievalQA`** — Enables retrieval-augmented generation (RAG) pipelines for question answering over financial documents.  
- **`RecursiveCharacterTextSplitter`** — Splits large text data (e.g., reports, filings) into manageable chunks.  
- **`Document`** — Data structure for storing text chunks with metadata.

### Financial Data APIs
- **`yfinance`** — Fetches historical and real-time stock market data.  
- **`NewsApiClient`** — Retrieves financial and market-related news articles.  
- **`Fred`** — Connects to the Federal Reserve Economic Data (FRED) API for macroeconomic indicators.  
- **`QueryApi` (SEC API)** — Accesses SEC filings for company fundamentals and disclosures.

### Numerical Computing
- **`numpy`** — Supports efficient numerical computations for analysis and model input preparation.

Together, these libraries enable **data collection**, **LLM-based reasoning**, **semantic retrieval**, and **financial analysis**, forming the backbone of the **agentic financial intelligence system**.


## Load Environment Variables

In [40]:
def load_env(filepath="config/aai_520_proj.config"):
    """
    Loads environment variables from the aai_520_project.config.
    Each line in the file should be in the format KEY=VALUE.
    """
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    os.environ[key] = value
        print(f"Environment variables loaded from {filepath}")
    except FileNotFoundError:
        print(f"Error: Config file not found at {filepath}. Make sure it is in the project config directory.")
    except Exception as e:
        print(f"Error loading environment variables from {filepath}: {e}")


### Function: `load_env()`

The `load_env()` function is used to **load environment variables** from a configuration file (default: `config/aai_520_proj.config`).  
Each line in the config file should follow the format:



## RAG Pipeline

In [41]:
class RAG_pipeline:
    def __init__(self,modelName, apiKey):
        self.modelName = modelName
        self.apiKey = apiKey

    def getLLM(self):
        # "gemini-2.5-flash"
        llm = ChatGoogleGenerativeAI(model=self.modelName, google_api_key=os.getenv(self.apiKey))
        return llm

    def getLLM_withlayers(self, context, prompt):
        content = ""
        # print(context)
        for i in context:
            # print(i)
            if(context[i]):
                content+=(i+":\n")
                content+=(str(context[i])+"\n")

        # print(content)
        docs = [Document(page_content=content)]

        # 2. Split docs into smaller chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = splitter.split_documents(docs)

        # 3. Create embeddings
        embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        # embeddings = [embedding_model.embed(chunk.page_content) for chunk in chunks]

        # 4. Store in vector database (FAISS)
        vector_store = FAISS.from_documents(chunks, embedding_model)

        return vector_store

## Class: `RAG_pipeline`

The `RAG_pipeline` class defines a **Retrieval-Augmented Generation (RAG)** framework that connects **Google’s Generative AI models** with **LangChain tools** for intelligent, context-aware reasoning.  
It allows large language models (LLMs) to leverage relevant external data — such as financial reports, market summaries, or company metrics — to generate more accurate and grounded responses.

---

### **1. Initialization**
#### Purpose:
Initializes the RAG pipeline with the model name and API key reference.
#### Parameters:
- modelName: The name of the generative AI model to be used (e.g., "gemini-2.5-flash").  
- apiKey: The environment variable name containing the API key required for authentication.  

When initialized, these parameters are stored as class attributes and used by subsequent methods to build the pipeline.

### **2. Method: getLLM()**
#### Purpose:
Creates and returns a Google Generative AI chat model instance for reasoning and text generation.
#### Details:
- Fetches the API key from environment variables.
- Initializes a ChatGoogleGenerativeAI object from LangChain.
- Serves as the core language model interface for downstream agents and query modules.
#### Returns:
A configured LLM instance ready for use in financial analysis, question answering, or decision support tasks.

### **3. Method: getLLM_withlayers(context, prompt)**
#### Purpose:
Builds a retrieval-augmented knowledge layer to support context-grounded responses from the LLM.
#### Detailed Workflow:
1. Context Assembly:
- Iterates through the provided context dictionary.
- Formats each key–value pair into structured text (e.g., company details, metrics, news).
- Wraps the compiled content into a LangChain Document object.
2. Text Chunking:
- Splits the document into smaller segments using RecursiveCharacterTextSplitter.
- This ensures optimal processing and embedding quality.
3. Embedding Generation:
- Transforms each text chunk into a high-dimensional vector using the Hugging Face Sentence Transformer model (all-mpnet-base-v2).
- These embeddings capture the semantic meaning of the content.
4. Vector Storage (FAISS Index):
- The generated embeddings are stored in a FAISS vector database.
- Enables fast and accurate similarity search during query retrieval.
#### Returns:
A FAISS vector store object containing the embedded text chunks — ready for use in a Retrieval-Augmented Generation (RAG) query pipeline.
#### Overall Purpose
The RAG_pipeline class acts as the foundation of retrieval-augmented reasoning within the multi-agent financial system.
By combining:
- LLM reasoning (via Google Generative AI), and
- Contextual data retrieval (via embeddings and FAISS),  

the pipeline enables agents to deliver grounded, explainable, and data-driven insights — essential for tasks such as financial document summarization, investment evaluation, and real-time market intelligence.

## LLM Integration

In [42]:
def call_gemini(system_instruction: str, user_prompt: str, json_output: bool = True) -> dict | str:
    """
    Calls the Gemini API with a system instruction and user prompt.

    Args:
        system_instruction: The system instruction for the model.
        user_prompt: The user's prompt.
        json_output: Whether to expect a JSON output from the model.

    Returns:
        A dictionary if json_output is True, otherwise a string.
    """


    genai.configure(
        api_key=os.environ.get('GOOGLE_API_KEY'),
    )

    model = genai.GenerativeModel(
        model_name=os.environ.get('GEMINI_MODEL_NAME'),
        generation_config={"response_mime_type": "application/json"} if json_output else None
    )

    prompt = f"{system_instruction}\n\n{user_prompt}"

    try:
        response = model.generate_content(prompt)
        if json_output:
            return json.loads(response.text)
        return response.text
    except Exception as e:
        print(f"An error occurred in call_gemini: {e}")
        return None

def call_judge_gemini(system_instruction: str, user_prompt: str, json_output: bool = True) -> dict | str:
    """
    Calls the Gemini API with a system instruction and user prompt for the judge model.

    Args:
        system_instruction: The system instruction for the model.
        user_prompt: The user's prompt.
        json_output: Whether to expect a JSON output from the model.

    Returns:
        A dictionary if json_output is True, otherwise a string.
    """

    genai.configure(
        api_key=os.environ.get('GOOGLE_API_KEY'),
    )

    model = genai.GenerativeModel(
        model_name=os.environ.get('JUDGE_MODEL_NAME'),
        generation_config={"response_mime_type": "application/json"} if json_output else None
    )

    prompt = f"{system_instruction}\n\n{user_prompt}"

    try:
        response = model.generate_content(prompt)
        if json_output:
            return json.loads(response.text)
        return response.text
    except Exception as e:
        print(f"An error occurred in call_judge_gemini: {e}")
        return None


## 🤖 LLM Integration

This section defines the interface for integrating **Google’s Gemini models** into the multi-agent financial analysis system.  
Two key functions — `call_gemini()` and `call_judge_gemini()` — enable structured communication with different Gemini model instances for task execution and evaluation.

---

### **1. `call_gemini()`**
This function connects to the **primary Gemini model** to handle core reasoning and generation tasks.  
It accepts both a **system instruction** (defining model behavior) and a **user prompt**, optionally returning the output in JSON format.

**Workflow:**
- Configures the Gemini API using the stored `GOOGLE_API_KEY`.  
- Initializes the model defined in the `GEMINI_MODEL_NAME` environment variable.  
- Combines system and user inputs into a single prompt.  
- Sends the prompt to Gemini for processing and returns the model’s response (either as parsed JSON or plain text).

### **2. `call_judge_gemini()`**
This function interacts with a secondary 'judge' Gemini model, designed to evaluate or critique outputs produced by other agents or models in the system.

**Workflow:**
- Configures the Gemini API similarly to `call_gemini()`.
- Uses the model specified in `JUDGE_MODEL_NAME`.
- Sends combined system and user instructions for assessment or validation.
- Returns the model’s structured evaluation in JSON or text form.

In [43]:
class AgentLogger:
    def __init__(self, state):
        self.state = state
        self.state.setdefault("conversation_logs", [])

    def log(self, sender, receiver, content, **metadata):
        self.state["conversation_logs"].append({
            "timestamp": datetime.utcnow().isoformat(),
            "sender": sender,
            "receiver": receiver,
            "content": content,
            "metadata": metadata or {}
        })

### Class: `AgentLogger`

The `AgentLogger` class is responsible for **tracking and recording interactions** between agents within the multi-agent financial system.

**Purpose:**  
Maintains a structured log of all agent communications, enabling transparency, traceability, and debugging of multi-agent workflows.

**Key Features:**
- Initializes with a shared `state` dictionary that stores conversation history.  
- Automatically creates a `"conversation_logs"` list if it doesn't exist.  
- The `log()` method records each message with:
  - `timestamp` — UTC time of the interaction.  
  - `sender` and `receiver` — Identifying the communicating agents.  
  - `content` — The message or data exchanged.  
  - `metadata` — Optional additional context or tags.

## Memory Agent

In [44]:
class MemoryAgent:
    def __init__(self, db_path='memory_db.json'):
        self.db_path = db_path
        self.memory = self._load_memory()

    # ------------------------------------------------------------
    # Internal helper to attach logger
    # ------------------------------------------------------------
    def _get_logger(self, state):
        return AgentLogger(state) if state and "conversation_logs" in state else None

    # ------------------------------------------------------------
    # Load memory from JSON file
    # ------------------------------------------------------------
    def _load_memory(self):
        try:
            if not os.path.exists(self.db_path):
                return {}
            with open(self.db_path, 'r') as f:
                return json.load(f)
        except Exception as e:
            print(f" Failed to load memory DB: {e}")
            return {}

    # ------------------------------------------------------------
    # Save memory to disk
    # ------------------------------------------------------------
    def _save_memory(self):
        try:
            with open(self.db_path, 'w') as f:
                json.dump(self.memory, f, indent=4)
        except Exception as e:
            print(f" Failed to save memory DB: {e}")

    # ------------------------------------------------------------
    # Retrieve stored memory
    # ------------------------------------------------------------
    def retrieve(self, symbol: str, state: dict = None) -> dict:
        """Retrieves memory for a given stock symbol."""
        logger = self._get_logger(state)
        try:
            memory_entry = self.memory.get(symbol)
            if memory_entry:
                if logger:
                    logger.log("MemoryAgent", "System", f"Retrieved memory for {symbol}")
                return memory_entry
            else:
                if logger:
                    logger.log("MemoryAgent", "System", f"No memory found for {symbol}")
                return None
        except Exception as e:
            error_details = traceback.format_exc()
            if logger:
                logger.log("MemoryAgent", "System",
                           f"Error retrieving memory for {symbol}: {e}",
                           level="error", traceback=error_details)
            return None

    # ------------------------------------------------------------
    # Update memory
    # ------------------------------------------------------------
    def update(self, symbol: str, final_analysis: dict, state: dict = None):
        """Updates or creates a memory entry for a stock symbol."""
        logger = self._get_logger(state)

        try:
            if symbol not in self.memory:
                self.memory[symbol] = {}

            self.memory[symbol] = {
                'summary': final_analysis.get('summary', ''),
                'key_metrics': final_analysis.get('key_metrics', {}),
                'date': datetime.now().isoformat()
            }

            self._save_memory()

            msg = f"Memory updated for {symbol}"
            print(msg)
            if logger:
                logger.log("MemoryAgent", "System", msg, payload=self.memory[symbol])

        except Exception as e:
            error_details = traceback.format_exc()
            print(f" Error updating memory for {symbol}: {e}")
            if logger:
                logger.log("MemoryAgent", "System",
                           f"Error updating memory for {symbol}: {e}",
                           level="error", traceback=error_details)



### 🧠 Class: `MemoryAgent`

The `MemoryAgent` is responsible for **persistent knowledge storage and retrieval** within the multi-agent financial analysis system.  
It enables agents to remember past analyses, store insights, and reuse relevant information for future decision-making — creating a form of **long-term memory** for the system.

---

### **Key Responsibilities**

1. **Memory Management**
   - Loads and saves agent memory in a local JSON database (`memory_db.json` by default).  
   - Ensures previous financial analyses and summaries are retained across sessions.

2. **Retrieval**
   - The `retrieve()` method fetches stored memory for a given stock symbol.  
   - If a record exists, it returns the stored summary and metrics; otherwise, it logs that no memory was found.  
   - Useful for recalling previous analyses and avoiding redundant computations.

3. **Update**
   - The `update()` method creates or updates a memory entry with:
     - `summary`: High-level overview of the financial analysis.  
     - `key_metrics`: Important extracted data points.  
     - `date`: Timestamp of when the entry was last updated.  
   - Automatically persists updates to disk for future retrieval.

4. **Logging Integration**
   - Uses the `AgentLogger` to record memory access and updates within the shared system `state`.  
   - Logs both normal operations and errors for transparency and debugging.

---

### **Internal Helpers**
- `_load_memory()`: Loads memory from the JSON file at initialization.  
- `_save_memory()`: Writes updated memory data back to disk.  
- `_get_logger(state)`: Attaches a logger if the shared conversation state is provided.

### Purpose
The MemoryAgent ensures continuity and context retention across multiple financial analyses.
By maintaining a persistent memory of previous results, it enables the agentic system to:
- Build cumulative intelligence over time,
- Reference past evaluations for trend detection, and
- Support iterative improvement and reasoning consistency across sessions.

## Financial Services Planning Agent

In [45]:
class PlanningAgent:
    def __init__(self):
        pass

    def generate_plan(self, symbol: str, state: dict, memory: str = None) -> list[str]:
        """Generates a research plan for a given stock symbol."""

        logger = AgentLogger(state)

        system_instruction = (
            "You are an expert investment analyst planning a research workflow. "
            "Given the stock symbol and the historical memory, generate a list of the 5-7 most critical steps "
            "(including tool calls and internal processes) to generate a final investment thesis. "
            "Output must be a JSON array of strings."
        )

        user_prompt = f"Stock Symbol: {symbol}"
        if memory:
            user_prompt += f"\n\nHistorical Memory:\n{memory}"

        # Log outgoing LLM request
        logger.log("PlanningAgent", "LLM", f"Requesting research plan for {symbol}...", prompt=user_prompt)
        # response = call_gemini(system_instruction, user_prompt, json_output=True)

        ragObject = RAG_pipeline("gemini-2.0-flash", "GOOGLE_API_KEY")
        chain = ragObject.getLLM()

        response = chain.invoke(system_instruction+user_prompt).content

        cleaned = response.strip("```json").strip("```").strip()
        response = json.loads(cleaned)

        if response and isinstance(response, list):
            logger.log("LLM", "PlanningAgent", f"Received plan: {response}")
            return response
        else:
            logger.log("PlanningAgent", "LLM", f"Invalid or empty response for {symbol}", level="error")
            print("Failed to generate a valid plan.")
            return []


### 🧭 Class: `PlanningAgent`

The `PlanningAgent` is responsible for **strategic task planning** within the multi-agent financial analysis system.  
It serves as the **orchestrator**, designing a structured research workflow for analyzing a given stock symbol — outlining which tools, agents, and steps are required to generate a comprehensive investment thesis.

---

### **Core Functionality**

#### **`generate_plan()`**
**Purpose:**
Generates a clear, step-by-step research plan for analyzing a company based on its stock symbol and optionally, its historical memory.  
**Workflow:**
1. Context Setup
- Uses a `system_instruction` that defines the role of the LLM as an investment research planner.  
- Requests 5–7 actionable steps as a JSON array outlining the complete analytical process.
2. Memory Utilization
- Incorporates previously stored insights (if available) from the `MemoryAgent` to create a context-aware and non-redundant plan.
3. LLM Integration
- Uses the RAG_pipeline to initialize a Gemini-based reasoning model (`gemini-2.0-flash`).
- Sends both system and user prompts to the model to generate the workflow steps.
4. Response Handling
- Parses and cleans the model’s JSON response.
- Returns the structured plan as a Python list of strings.
- Logs all interactions (requests, responses, and errors) using the `AgentLogger.`

## Prompt Chaining Agent

In [46]:
class PromptChainingAgent:
    def __init__(self):
        pass

    def _get_logger(self, state):
        """Attach logger to agent if available."""
        return AgentLogger(state) if state and "conversation_logs" in state else None

    def run(self, raw_text: str, state: dict = None) -> dict:
        """Runs a 5-stage prompt chain to process raw text with detailed logging."""

        logger = self._get_logger(state)
        results = {}

        try:
            # --------------------------------------------------------------------------------
            # Stage 1: Ingest / Preprocess
            # --------------------------------------------------------------------------------
            preprocess_prompt = f"Clean the following text and remove any boilerplate content:\n\n{raw_text}"
            if logger:
                logger.log("PromptChainingAgent", "System", "Stage 1: Preprocessing text input.")
            clean_text = call_gemini("You are a text cleaning assistant.", preprocess_prompt, json_output=False)

            if not clean_text:
                msg = "Failed to clean text."
                if logger:
                    logger.log("PromptChainingAgent", "System", msg, level="error")
                return {"error": msg}

            if logger:
                logger.log("PromptChainingAgent", "System", "Text cleaned successfully.", payload={"clean_text": clean_text[:500]})
            print("\n--- Cleaned Text ---")
            print(clean_text)

            # --------------------------------------------------------------------------------
            # Stage 2: Classification
            # --------------------------------------------------------------------------------
            classify_prompt = f"What is the primary event type in this text? (e.g., Earnings, Product Launch, Regulation, Macro):\n\n{clean_text}"
            if logger:
                logger.log("PromptChainingAgent", "System", "Stage 2: Classifying text.")
            classification = call_gemini("You are a text classification specialist.", classify_prompt, json_output=False)

            if not classification:
                msg = "Failed to classify text."
                if logger:
                    logger.log("PromptChainingAgent", "System", msg, level="error")
                return {"error": msg}

            if logger:
                logger.log("PromptChainingAgent", "System", "Classification complete.", payload={"classification": classification.strip()})
            print(f"\n--- Classification ---\n{classification}")
            results["classification"] = classification.strip()

            # --------------------------------------------------------------------------------
            # Stage 3: Extraction
            # --------------------------------------------------------------------------------
            extract_prompt = f"Extract all numerical data points (e.g., EPS, Revenue, Guidance) mentioned in the text:\n\n{clean_text}"
            if logger:
                logger.log("PromptChainingAgent", "System", "Stage 3: Extracting numerical data.")
            extracted_data = call_gemini("You are a data extraction expert.", extract_prompt, json_output=True)

            if not extracted_data:
                msg = "Failed to extract data."
                if logger:
                    logger.log("PromptChainingAgent", "System", msg, level="error")
                return {"error": msg}

            if logger:
                logger.log("PromptChainingAgent", "System", "Data extraction complete.", payload={"extracted_data": extracted_data})
            print(f"\n--- Extracted Data ---\n{extracted_data}")
            results["extracted_data"] = extracted_data

            # --------------------------------------------------------------------------------
            # Stage 4: Summarization
            # --------------------------------------------------------------------------------
            summarize_prompt = f"Write a concise, abstractive summary of the key market takeaway (1-2 sentences):\n\n{clean_text}"
            if logger:
                logger.log("PromptChainingAgent", "System", "Stage 4: Summarizing content.")
            summary = call_gemini("You are a financial news summarizer.", summarize_prompt, json_output=False)

            if not summary:
                msg = "Failed to summarize text."
                if logger:
                    logger.log("PromptChainingAgent", "System", msg, level="error")
                return {"error": msg}

            if logger:
                logger.log("PromptChainingAgent", "System", "Summary complete.", payload={"summary": summary.strip()})
            print(f"\n--- Summary ---\n{summary}")
            results["summary"] = summary.strip()

            # --------------------------------------------------------------------------------
            # Final Results
            # --------------------------------------------------------------------------------
            if logger:
                logger.log("PromptChainingAgent", "System", "Prompt chaining complete.", payload=results)
            return results

        except Exception as e:
            error_details = traceback.format_exc()
            if logger:
                logger.log("PromptChainingAgent", "System",
                           f"Unhandled exception in prompt chain: {e}",
                           level="error",
                           traceback=error_details)
            return {"error": f"Unhandled exception: {e}"}



### Class: `PromptChainingAgent`

The `PromptChainingAgent` is designed to **process raw text through a structured multi-stage prompt workflow**.  
It leverages LLMs (via Gemini) to perform **text cleaning, classification, data extraction, and summarization** in a chained, modular fashion — producing actionable outputs for financial analysis.

### **Core Functionality**

#### **1. `_get_logger(state)`**
- Attaches an `AgentLogger` instance if a shared `state` is provided.  
- Enables detailed logging of each stage in the prompt chain for traceability.

#### **2. `run(raw_text, state)`**
Processes the input text in **five key stages**:

1. **Preprocessing**
   - Cleans and normalizes the input text.  
   - Removes boilerplate content to prepare for downstream tasks.  
   - Logs the process and any errors.

2. **Classification**
   - Determines the primary event type in the text (e.g., Earnings, Product Launch, Regulation, Macro).  
   - Supports contextual tagging and workflow routing for agentic analysis.

3. **Data Extraction**
   - Extracts all numerical data points (EPS, Revenue, Guidance, etc.) from the text.  
   - Produces structured output suitable for RAG or memory storage.

4. **Summarization**
   - Generates a concise, abstractive 1–2 sentence summary of the key insights or market takeaway.  
   - Provides a high-level overview for rapid consumption.

5. **Final Aggregation**
   - Combines classification, extracted data, and summary into a single results dictionary.  
   - Logs the completed pipeline for auditing and debugging.

### **Logging**
- Every stage of the prompt chain is logged via `AgentLogger`, including:
  - Stage start and completion
  - Payload details (e.g., cleaned text, classification, extracted data)
  - Errors or exceptions


## Routing Agent

In [47]:
class RoutingAgent:
    def __init__(self):
        pass

    def _get_logger(self, state):
        """Attach logger if conversation state is provided."""
        return AgentLogger(state) if state and "conversation_logs" in state else None

    def route(self, classification: str, state: dict = None) -> str:
        """Determines the next agent path based on classification with structured logging."""
        logger = self._get_logger(state)

        try:
            if not classification or not isinstance(classification, str):
                msg = "Invalid or empty classification received."
                if logger:
                    logger.log("RoutingAgent", "System", msg, level="error")
                return "GeneralAnalysis"

            normalized_class = classification.lower().strip()
            if logger:
                logger.log(
                    "RoutingAgent",
                    "System",
                    f"Received classification: '{classification}'",
                    payload={"normalized_class": normalized_class}
                )

            if 'earnings' in normalized_class:
                route = 'EarningsModelRun'
            elif 'regulation' in normalized_class:
                route = 'ComplianceCheck'
            elif 'product launch' in normalized_class or 'launch' in normalized_class:
                route = 'MarketImpactAnalysis'
            else:
                route = 'GeneralAnalysis'

            if logger:
                logger.log(
                    "RoutingAgent",
                    "System",
                    f"Routing decision: {route}",
                    payload={"classification": classification, "next_route": route}
                )

            return route

        except Exception as e:
            error_details = traceback.format_exc()
            if logger:
                logger.log("RoutingAgent", "System",
                           f"Routing error: {e}",
                           level="error",
                           traceback=error_details)
            return "GeneralAnalysis"



### Class: `RoutingAgent`

The `RoutingAgent` is responsible for **determining the appropriate workflow or agent path** based on the classification of financial text or events.  
It acts as a **decision router** in the multi-agent system, ensuring that tasks are sent to the most suitable specialized agent for further processing.

### **Core Functionality**

#### **1. `_get_logger(state)`**
- Attaches an `AgentLogger` if a shared `state` with conversation logs is provided.  
- Enables structured logging of routing decisions and errors.

#### **2. `route(classification, state)`**
- Takes a text classification (e.g., "Earnings", "Regulation", "Product Launch") as input.  
- Determines the **next agent or process path** based on predefined rules:
  - `'earnings'` → `'EarningsModelRun'`  
  - `'regulation'` → `'ComplianceCheck'`  
  - `'product launch'` or `'launch'` → `'MarketImpactAnalysis'`  
  - Anything else → `'GeneralAnalysis'`  
- Returns the name of the next agent or workflow.

**Logging:**
- Logs the received classification, normalized value, and the routing decision.  
- Records errors and falls back to `'GeneralAnalysis'` if issues occur.

## Toolbox Agent

In [48]:
class ToolboxAgent:
    def __init__(self):
        self.cache = {}
        self.newsapi = NewsApiClient(api_key=os.environ.get('NEWS_API_KEY'))
        self.fred = Fred(api_key=os.environ.get('FRED_API_KEY'))
        self.sec = QueryApi(api_key=os.environ.get('SEC_API_KEY'))

    def _is_cache_valid(self, symbol, tool_name):
        if symbol in self.cache and tool_name in self.cache[symbol]:
            timestamp = self.cache[symbol][tool_name]['timestamp']
            if datetime.now() - timestamp < timedelta(hours=24):
                return True
        return False

    # Helper to initialize logger only once per symbol/session
    def _get_logger(self, state):
        return AgentLogger(state)

    # -----------------------------------------------------------------------------------
    # YFinance Data
    # -----------------------------------------------------------------------------------
    def get_yahoo_finance_data(self, symbol: str, state: dict) -> dict:
        """Fetches price, P/E, and fundamental metrics from Yahoo Finance."""
        tool_name = 'yfinance'
        logger = self._get_logger(state)

        if self._is_cache_valid(symbol, tool_name):
            print(f"Returning cached data for {symbol} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Cache hit for {symbol}")
            return self.cache[symbol][tool_name]['data']

        try:
            print(f"Fetching data for {symbol} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Fetching Yahoo Finance data for {symbol}")
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if symbol not in self.cache:
                self.cache[symbol] = {}
            self.cache[symbol][tool_name] = {
                'timestamp': datetime.now(),
                'data': info
            }

            logger.log(tool_name, "ToolboxAgent", f"Successfully fetched data for {symbol}")
            return info
        except Exception as e:
            error_details = traceback.format_exc()
            logger.log("ToolboxAgent", tool_name, f"Error fetching yfinance data for {symbol}: {e}", level="error", traceback=error_details)
            print(f" YFinance Error for {symbol}: {e}")
            return None

    # -----------------------------------------------------------------------------------
    # Financial News
    # -----------------------------------------------------------------------------------
    def get_financial_news(self, symbol: str, state: dict) -> dict:
        """Fetches financial news for a given symbol."""
        tool_name = 'newsapi'
        logger = self._get_logger(state)

        if self._is_cache_valid(symbol, tool_name):
            print(f"Returning cached news for {symbol}")
            logger.log("ToolboxAgent", tool_name, f"Cache hit for {symbol} (news)")
            return self.cache[symbol][tool_name]['data']

        try:
            print(f"Fetching news for {symbol}")
            logger.log("ToolboxAgent", tool_name, f"Fetching news for {symbol}")
            all_articles = self.newsapi.get_everything(
                q=symbol,
                language='en',
                sort_by='relevancy',
                page_size=5
            )
            if symbol not in self.cache:
                self.cache[symbol] = {}
            self.cache[symbol][tool_name] = {
                'timestamp': datetime.now(),
                'data': all_articles
            }

            logger.log(tool_name, "ToolboxAgent", f"Fetched {len(all_articles.get('articles', []))} news articles for {symbol}")
            return all_articles

        except Exception as e:
            error_details = traceback.format_exc()
            print(f" NewsAPI Error for {symbol}: {e}")
            logger.log("ToolboxAgent", tool_name, f"Error fetching news for {symbol}: {e}", level="error", traceback=error_details)
            return None

    # -----------------------------------------------------------------------------------
    # Economic Data
    # -----------------------------------------------------------------------------------
    def get_economic_data(self, indicator: str, state: dict) -> dict:
        """Fetches economic data from FRED."""
        tool_name = 'fred'
        logger = self._get_logger(state)
        if self._is_cache_valid(indicator, tool_name):
            print(f"Returning cached data for {indicator} from {tool_name}")
            return self.cache[indicator][tool_name]['data']

        try:
            print(f"Fetching data for {indicator} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Fetching economic data for indicator '{indicator}'")
            data = self.fred.get_series(indicator)

            logger.log(tool_name, "ToolboxAgent", f"Successfully fetched {len(data)} records for {indicator}")

            if indicator not in self.cache:
                self.cache[indicator] = {}
            self.cache[indicator][tool_name] = {
                'timestamp': datetime.now(),
                'data': data.to_dict()
            }
            return data.to_dict()
        except Exception as e:
            error_details = traceback.format_exc()
            print(f"An error occurred with FRED for indicator {indicator}: {e}")
            logger.log("ToolboxAgent", tool_name, f"Error fetching FRED data for {indicator}: {e}", level="error", traceback=error_details)
            return None

    # -----------------------------------------------------------------------------------
    # Filing Data (SEC EDGAR)
    # -----------------------------------------------------------------------------------
    def get_filing_data(self, indicator: str, state: dict) -> dict:
        """Fetches Filings data from Sec Edgar."""
        tool_name = 'secEdgar'
        logger = self._get_logger(state)

        query = {
            "query": (
                f'(formType:"10-K" OR formType:"10-Q" OR formType:"8-K" OR '
                f'formType:"SC 13D" OR formType:"SC 13G") AND ticker:{indicator}'
            ),
            "from": 0,
            "size": 4,
            "sort": [{"filedAt": {"order": "desc"}}]
        }
        # print(query)

        logger.log("ToolboxAgent", tool_name, f"Preparing SEC EDGAR query for {indicator}", query=query)

        # Check cache first
        if self._is_cache_valid(indicator, tool_name):
            print(f"Returning cached data for {indicator} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Cache hit for SEC EDGAR filings of {indicator}")
            return self.cache[indicator][tool_name]['data']

        try:
            print(f"Fetching data for {indicator} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Fetching latest SEC filings for {indicator}")
            data = self.sec.get_filings(query)["filings"]

            filingDataRaw = {}
            folder_path = os.path.join("..", "utils", "filingDocuments", indicator)
            os.makedirs(folder_path, exist_ok=True)

            for filing in data:
                folder_path = "..\\utils\\filingDocuments\\"+(indicator)
                os.makedirs(folder_path, exist_ok=True)
                form_type = filing["formType"].replace("/", "-")
                description = filing["description"].replace("/", "-")

                fileType = {}

                for doc in filing.get("documentFormatFiles", []):
                    doc_url = doc.get("documentUrl", "")
                    if not doc_url:
                        continue

                    file_ext = os.path.splitext(doc_url)[1]
                    file_name = f"{form_type}-{description}{file_ext}"
                    file_path = os.path.join(folder_path, file_name)

                    if file_ext in [".txt", ".htm", ".html"]:
                        try:
                            response = requests.get(doc_url, timeout=10)
                            response.raise_for_status()
                            with open(file_path, "wb") as f:
                                f.write(response.content)
                            filingDataRaw[file_name] = response.content.decode("utf-8", errors="ignore")

                            logger.log(tool_name, "ToolboxAgent", f"Saved filing {file_name} for {indicator}")
                        except Exception as e:
                            error_details = traceback.format_exc()
                            logger.log("ToolboxAgent", tool_name,
                                       f"Error downloading {file_name} for {indicator}: {e}",
                                       level="error", traceback=error_details)
                    else:
                        logger.log("ToolboxAgent", tool_name, f"Skipping unsupported file type: {file_ext}")


            # Update cache after successful fetch
            if indicator not in self.cache:
                self.cache[indicator] = {}
            self.cache[indicator][tool_name] = {
                'timestamp': datetime.now(),
                'data': filingDataRaw
            }
            logger.log(tool_name, "ToolboxAgent", f"Fetched and cached {len(filingDataRaw)} filings for {indicator}")
            return filingDataRaw
        except Exception as e:
            error_details = traceback.format_exc()
            print(f" SEC EDGAR Error for {indicator}: {e}")
            logger.log("ToolboxAgent", tool_name,
                       f"Error fetching filings for {indicator}: {e}",
                       level="error", traceback=error_details)
            return None

    def fetch(self, tool_name: str, symbol: str, state: dict) -> dict:
        """Dynamically dispatches to the correct tool wrapper."""
        logger = self._get_logger(state)
        if tool_name == 'yfinance':
            return self.get_yahoo_finance_data(symbol, state)
        elif tool_name == 'newsapi':
            return self.get_financial_news(symbol, state)
        elif tool_name == 'fred':
            return self.get_economic_data(symbol, state)
        elif tool_name == 'secEdgar':
            return self.get_filing_data(symbol, state)
        else:
            print(f"Tool {tool_name} not recognized.")
            logger.log(tool_name, "ToolboxAgent", f"Tool {tool_name} not recognized")
            return None


### Class: `ToolboxAgent`

The `ToolboxAgent` provides a **centralized toolkit for fetching and caching financial data** from multiple sources.  
It acts as the **data retrieval layer** in the multi-agent financial system, enabling other agents to access structured information efficiently.

### **Core Functionality**

1. **Initialization**
   - Sets up API clients for:
     - **Yahoo Finance** (`yfinance`)  
     - **NewsAPI** (`newsapi`)  
     - **Federal Reserve Economic Data** (`FRED`)  
     - **SEC EDGAR Filings** (`sec_api`)  
   - Maintains an internal **cache** to store fetched data for 24 hours, reducing redundant requests.

2. **Caching**
   - `_is_cache_valid(symbol, tool_name)` checks if cached data is still fresh.  
   - Automatically returns cached results if valid.

3. **Logging**
   - `_get_logger(state)` attaches an `AgentLogger` to track data fetching events, errors, and cache hits.

### **Tool Methods**

- **`get_yahoo_finance_data(symbol, state)`**  
  Fetches stock price, P/E ratio, and other fundamental metrics from Yahoo Finance.

- **`get_financial_news(symbol, state)`**  
  Retrieves the most recent relevant news articles for a given symbol using NewsAPI.

- **`get_economic_data(indicator, state)`**  
  Fetches economic indicators (e.g., unemployment, CPI) from FRED.

- **`get_filing_data(indicator, state)`**  
  Retrieves SEC filings (10-K, 10-Q, 8-K, SC 13D/G) for a company, downloads documents, and caches the content locally.

- **`fetch(tool_name, symbol, state)`**  
  Dynamically dispatches requests to the appropriate tool method based on `tool_name`.

### **Purpose**
The `ToolboxAgent` enables **reliable, centralized access to diverse financial data sources** with:
- Automatic **caching** for efficiency  
- **Structured logging** for traceability  
- **Support for multiple data types** (stock metrics, news, economic indicators, filings)  

This allows the multi-agent system to gather, analyze, and integrate data seamlessly for financial research and decision-making.


## Evaluator Agent

In [49]:
class MultiAgentEvaluator:
    def __init__(self):
        self.openai_model = "gpt-4o"
        self.ollama_model = "llama2"
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")

        # Initialize OpenAI client only if key is set
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key and OpenAI is not None:
            try:
                self.client = OpenAI(api_key=api_key)
                self.mode = "openai"
            except Exception:
                self.client = None
                self.mode = "ollama"
        else:
            self.client = None
            self.mode = "ollama"

        print(f"Evaluator initialized in {self.mode.upper()} mode")

    def llm_grade(self, thesis: str, reference: str = None) -> dict:
        """Evaluate investment thesis quality using OpenAI or Ollama."""
        prompt = f"""
        Evaluate this investment thesis for clarity, factual accuracy, and rigor.
        Rate each dimension from 1–10 and summarize with justification.

        Thesis:
        {thesis}

        Reference (if provided):
        {reference}
        """

        # --- Try OpenAI first ---
        if self.mode == "openai" and self.client is not None:
            try:
                response = self.client.chat.completions.create(
                    model=self.openai_model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.2,
                )
                return {"source": "openai", "raw": response.choices[0].message.content}
            except Exception as e:
                print(f"[OpenAI Error] {e} — Falling back to Ollama.")
                self.mode = "ollama"

        # --- Fallback to Ollama ---
        if ollama is None:
            return {"error": "Neither OpenAI nor Ollama available."}

        try:
            response = ollama.chat(
                model=self.ollama_model,
                messages=[{"role": "user", "content": prompt}],
            )
            return {"source": "ollama", "raw": response["message"]["content"]}
        except Exception as e:
            return {"error": f"Both evaluators failed: {e}"}

    def embedding_consistency(self, thesis_a: str, thesis_b: str) -> float:
        """Measure semantic similarity between two analyses."""
        embeddings = self.embedder.encode([thesis_a, thesis_b])
        return cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

    def coordination_efficiency(self, logs: list) -> dict:
        """Analyze inter-agent message structure."""
        n_messages = len(logs)
        avg_message_len = np.mean([len(m["content"]) for m in logs])
        return {"n_messages": n_messages, "avg_message_len": avg_message_len}

### Class: `MultiAgentEvaluator`

The `MultiAgentEvaluator` is designed to **assess the quality, consistency, and efficiency of outputs** generated by the multi-agent financial system.  
It provides tools for **evaluating investment theses, measuring semantic similarity, and analyzing agent communication patterns**.

### **Core Functionality**

1. **Initialization**
   - Sets up LLM evaluation clients:
     - **OpenAI GPT-4o** if `OPENAI_API_KEY` is available.  
     - **Ollama LLaMA2** as fallback.  
   - Initializes a **sentence embedding model** (`all-MiniLM-L6-v2`) for semantic similarity calculations.  
   - Prints the active evaluation mode (`OPENAI` or `OLLAMA`).

2. **`llm_grade(thesis, reference)`**
   - Evaluates an investment thesis for:
     - **Clarity**  
     - **Factual accuracy**  
     - **Rigor**  
   - Accepts an optional reference for comparison.  
   - Returns the LLM-generated evaluation and justification.  
   - Falls back to Ollama if OpenAI evaluation fails.

3. **`embedding_consistency(thesis_a, thesis_b)`**
   - Computes the **semantic similarity** between two theses using sentence embeddings.  
   - Returns a cosine similarity score between 0 and 1, indicating consistency of analysis.

4. **`coordination_efficiency(logs)`**
   - Analyzes agent interaction logs to assess workflow efficiency.  
   - Returns metrics such as:
     - `n_messages` — total messages exchanged  
     - `avg_message_len` — average message length  
   - Helps identify communication bottlenecks or verbosity in agent coordination.

### **Purpose**
The `MultiAgentEvaluator` ensures **quality control and performance measurement** within the multi-agent financial system by:
- Quantifying the **accuracy and clarity** of generated investment analyses.  
- Measuring **consistency between agent outputs**.  
- Evaluating **coordination efficiency** across inter-agent communications.  

This agent provides a structured framework for **continuous improvement, auditability, and reliability** of automated financial reasoning.


## Final Thesis Optimizer Agent

In [50]:
class EvaluatorOptimizerAgent:
    def __init__(self):
        pass

    def _get_logger(self, state):
        """Attach logger to agent if state has conversation logs."""
        return AgentLogger(state) if state and "conversation_logs" in state else None

    def run(self, data: dict, state: dict = None) -> str:
        """Runs the evaluator-optimizer workflow with detailed logging."""

        logger = self._get_logger(state)
        try:
            # --------------------------------------------------------------------------------
            # 1. Optimizer Stage — Draft Thesis
            # --------------------------------------------------------------------------------
            draft_prompt = (
                "Generate a comprehensive draft investment analysis and thesis "
                "(Buy/Hold/Sell) based on the following data.\n\nData:\n"
                f"{data}"
            )
            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Stage 1: Generating initial draft thesis.")

            ragObject = RAG_pipeline("gemini-2.0-flash", "GOOGLE_API_KEY")
            dbVector = ragObject.getLLM_withlayers(data, draft_prompt)

            docs = dbVector.similarity_search("Invetsment Outlook", k=5)
            st = "\n\n".join([doc.page_content for doc in docs])

            draft_prompt = (
                "Generate a comprehensive draft investment analysis and thesis "
                "(Buy/Hold/Sell) based on the following data.\n\nData:\n"
                f"{st}"
            )
            draft = call_gemini("You are a financial analyst drafting an investment thesis.", draft_prompt, json_output=False)

            if not draft:
                msg = "Failed to generate a draft."
                if logger:
                    logger.log("EvaluatorOptimizerAgent", "System", msg, level="error")
                return msg

            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Draft thesis generated successfully.",
                           payload={"draft": draft[:500]})
            print("\n--- Initial Draft ---")
            print(draft)

            # --------------------------------------------------------------------------------
            # 2. Evaluator Stage — Critique Draft
            # --------------------------------------------------------------------------------
            evaluator_prompt = (
                "Critique the following investment draft for two things:\n"
                "1. Factual consistency (do the numbers match the source data?)\n"
                "2. Logical consistency (is the 'Buy' recommendation justified by the identified risks?).\n"
                "Provide a specific suggestion for refinement.\n\n"
                f"Draft:\n{draft}"
            )
            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Stage 2: Evaluating draft for consistency and logic.")
            critique = call_gemini("You are a meticulous financial evaluator.", evaluator_prompt, json_output=False)

            if not critique:
                msg = "Failed to generate a critique."
                if logger:
                    logger.log("EvaluatorOptimizerAgent", "System", msg, level="error")
                return msg

            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Critique generated successfully.",
                           payload={"critique": critique[:500]})
            print("\n--- Critique ---")
            print(critique)

            # --------------------------------------------------------------------------------
            # 3. Optimizer Stage — Refinement
            # --------------------------------------------------------------------------------
            refinement_prompt = (
                "Based on the critique provided, refine and correct the initial draft. "
                "Produce the final, polished investment thesis.\n\n"
                f"Initial Draft:\n{draft}\n\nCritique:\n{critique}"
            )
            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Stage 3: Refining draft based on critique.")
            final_thesis = call_gemini("You are a financial analyst refining your work.", refinement_prompt, json_output=False)

            if not final_thesis:
                msg = "Failed to generate the final thesis."
                if logger:
                    logger.log("EvaluatorOptimizerAgent", "System", msg, level="error")
                return msg

            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Final polished thesis generated successfully.",
                           payload={"final_thesis": final_thesis[:500]})
            print("\n--- Final Thesis ---")
            print(final_thesis)

            return final_thesis

        except Exception as e:
            error_details = traceback.format_exc()
            if logger:
                logger.log("EvaluatorOptimizerAgent", "System",
                           f"Unhandled exception in evaluator-optimizer pipeline: {e}",
                           level="error",
                           traceback=error_details)
            print(f"Unhandled exception: {e}")
            return f"Unhandled exception: {e}"



### Class: `EvaluatorOptimizerAgent`

The `EvaluatorOptimizerAgent` is responsible for **generating, evaluating, and refining investment theses** in a structured, multi-stage workflow.  
It combines **drafting, automated evaluation, and iterative optimization** to produce polished, high-quality investment recommendations.

### **Core Functionality**

1. **Initialization**
   - No special parameters are required for initialization.
   - Logging is dynamically attached via `AgentLogger` if a conversation `state` is provided.

2. **`run(data, state)`**
   Executes a **three-stage evaluator-optimizer pipeline**:

   **Stage 1 — Draft Thesis**
   - Generates an initial investment analysis and thesis (`Buy/Hold/Sell`) based on input financial data.  
   - Uses a **RAG pipeline** to retrieve relevant contextual information for drafting.  
   - Logs progress and any errors.

   **Stage 2 — Evaluate Draft**
   - Critiques the initial draft for:
     - **Factual consistency** — Are the numbers correct?  
     - **Logical consistency** — Is the recommendation justified by identified risks?  
   - Provides detailed suggestions for improvement.  
   - Logs the critique process and errors if any.

   **Stage 3 — Refine Draft**
   - Refines and corrects the initial draft using the critique feedback.  
   - Produces a **final, polished investment thesis** ready for downstream usage.  
   - Logs the final output and ensures traceability.

### **Logging**
- Each stage logs:
  - Start and completion messages  
  - Key payload snippets (e.g., draft, critique, final thesis)  
  - Errors and exceptions with traceback details  

This ensures **full auditability of the evaluator-optimizer workflow**.

### **Purpose**
The `EvaluatorOptimizerAgent` enables a **high-quality, iterative investment analysis workflow**:
- Automates thesis generation from raw data  
- Ensures **factual and logical accuracy**  
- Produces **polished, actionable recommendations** for financial decision-making  
- Integrates seamlessly into a multi-agent financial reasoning system.


## Analysis Function

In [51]:
def run_analysis(symbol: str):
    """Runs the full agentic analysis for a given stock symbol."""

    # Load API keys and configure Gemini
    load_env()
    genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

    # 1. Initialize Agents
    toolbox = ToolboxAgent()
    memory = MemoryAgent()
    planner = PlanningAgent()
    prompt_chainer = PromptChainingAgent()
    router = RoutingAgent()
    evaluator = EvaluatorOptimizerAgent()

    # 2. Define State
    state = {
        "symbol": symbol,
        "plan": [],
        "raw_data": {},
        "processed_news": [],
        "conversational_logs": [],
        "final_thesis": None
    }

    print(f"--- Starting Analysis for {symbol} ---")

    # 3. Establish Flow
    # Input Symbol -> Memory Agent -> Planning Engine Agent
    retrieved_memory = memory.retrieve(symbol, state)
    if retrieved_memory:
        print(f"\n--- Retrieved Memory for {symbol} ---")
        print(json.dumps(retrieved_memory, indent=4))

    state["plan"] = planner.generate_plan(symbol, state, json.dumps(retrieved_memory) if retrieved_memory else None)
    if not state["plan"]:
        print("Could not generate a plan. Exiting.")
        return

    print(f"\n--- Generated Plan for {symbol} ---")
    for step in state["plan"]:
        print(f"- {step}")

    # The sequence then calls the Toolbox Agent multiple times (Uncomment from here)
    for step in state["plan"]:
        if ("assessment" in step.lower() or "analysis" in step.lower()) and 'yfinance' not in state["raw_data"]:
            state["raw_data"]['yfinance'] = toolbox.fetch('yfinance', symbol, state)
        if ("news" in step.lower() or "finding" in step.lower() or "analysis" in step.lower()) and 'news' not in state["raw_data"]:
            state["raw_data"]['news'] = toolbox.fetch('newsapi', symbol, state)
        if ("economic" in step.lower() or "advancements" in step.lower()) and 'fred_gdp' not in state["raw_data"]:
            # A more robust implementation would parse the indicator
            state["raw_data"]['fred_gdp'] = toolbox.fetch('fred', 'GDP', state)
        if ("valuation" in step.lower() or "risk" in step.lower() or "report" in step.lower()) and "secEdgar" not in state["raw_data"]:
            # A more robust implementation would parse the indicator
            state["raw_data"]['secEdgar'] = toolbox.fetch('secEdgar', symbol, state)

    print("\n--- Fetched Raw Data ---")
    # Abridged printing for brevity
    if 'yfinance' in state["raw_data"]:
        print("  - Yahoo Finance data retrieved.")
    if 'news' in state["raw_data"]:
        print("  - News data retrieved.")
    if 'fred_gdp' in state["raw_data"]:
        print("  - FRED GDP data retrieved.")
    if 'secEdgar' in state["raw_data"]:
        print("  - Sec Edgar data retrieved.")

    # Toolbox Output -> Prompt Chaining Agent -> Routing Agent
    if 'news' in state["raw_data"] and state["raw_data"]['news']!=None and state["raw_data"]['news']['articles']:
        for article in state["raw_data"]['news']['articles']:
            processed_article = prompt_chainer.run(article['title'] + "\n" + article.get('description', ''), state)
            state["processed_news"].append(processed_article)

            state["classification"] = router.route(processed_article.get('classification', ''), state)

            print(f"\n--- Routing for article: '{article['title']}' ---")
            print(f"  - Classification: {processed_article.get('classification')}")
            print(f"  - Route: {state["classification"]}")

            # Routing -> Execution of Specialized Model (Placeholder)

    # All data -> Evaluator–Optimizer Agent
    print("\n--- Generating Final Thesis with Evaluator-Optimizer ---")

    # Collect all relevant structured data for evaluation
    evaluator_data = {
        "symbol": state.get("symbol"),
        "classification": state.get("classification"),
        "financials": state.get("raw_data", {}).get("yfinance", []),
        "news": state.get("processed_news"),
        "economics": state.get("raw_data", {}).get("fred_gdp", []),
        "filings": state.get("raw_data", {}).get("secEdgar", [])
    }

    state["final_thesis"] = evaluator.run(evaluator_data, state)

    final_thesis = state["final_thesis"]
    logs = state.get("conversation_logs", [])

    evaluator = MultiAgentEvaluator()

    # LLM-based evaluation
    eval_result = evaluator.llm_grade(final_thesis)
    # Coordination metrics
    coordination = evaluator.coordination_efficiency(logs)

    # Basic heuristic parsing of scores from LLM text
    eval_text = eval_result.get("raw", "")
    clarity = accuracy = rigor = overall = 0

    # Regex patterns to extract scores
    patterns = {
        "clarity": r"clarity[:\s]*([0-9]+)\s*/\s*10",
        "accuracy": r"accuracy[:\s]*([0-9]+)\s*/\s*10",
        "rigor": r"rigor[:\s]*([0-9]+)\s*/\s*10",
        "overall": r"overall.*?([0-9]+)\s*/\s*10"
    }

    # Extract scores
    for key, pattern in patterns.items():
        match = re.search(pattern, eval_text, re.IGNORECASE)
        if match:
            score = int(match.group(1))
            if key == "clarity":
                clarity = score
            elif key == "accuracy":
                accuracy = score
            elif key == "rigor":
                rigor = score
            elif key == "overall":
                overall = score

    eval_metrics = {
        "clarity": clarity,
        "accuracy": accuracy,
        "rigor": rigor,
        "overall": overall,
        "source": eval_result.get("source", "unknown"),
        "evaluation_summary": eval_text,
    }

    state["evaluation"] = eval_metrics

    print("\n--- Evaluation Metrics ---")
    print(eval_metrics)

    memory.update(symbol, state["evaluation"])

    # Evaluator–Optimizer Output -> Memory Agent (Update)
    if state["final_thesis"]:
        # A more robust implementation would extract key metrics from the thesis
        memory.update(symbol, {"summary": state["final_thesis"]}, state)

        print(f"\n--- Completed Analysis for {symbol} ---")
        print("Final Thesis:")
        print(state["final_thesis"])

        return state


    return f"No summary generated for the symbol: {symbol}"

### Function: `run_analysis(symbol: str)`

The `run_analysis` function orchestrates the **full agentic financial analysis workflow** for a given stock symbol, leveraging the multi-agent system.

### **Workflow Overview**

1. **Environment Setup**
   - Loads API keys using `load_env()`.
   - Configures the Gemini API client (`genai`) for LLM interactions.

2. **Agent Initialization**
   - **ToolboxAgent:** Fetches financial data, news, economic indicators, and SEC filings.
   - **MemoryAgent:** Retrieves and updates historical analysis data.
   - **PlanningAgent:** Generates a research plan for the stock symbol.
   - **PromptChainingAgent:** Processes and extracts structured insights from raw news content.
   - **RoutingAgent:** Determines the appropriate downstream agent/model based on text classification.
   - **EvaluatorOptimizerAgent:** Generates, evaluates, and refines the final investment thesis.

3. **State Definition**
   - Maintains a structured `state` dictionary to track:
     - Symbol, research plan, raw and processed data, conversation logs, and final thesis.

4. **Memory Retrieval & Planning**
   - Retrieves historical memory for the symbol (if available).
   - Generates a multi-step research plan using the PlanningAgent.

5. **Data Fetching via ToolboxAgent**
   - Fetches relevant data according to the research plan:
     - **Yahoo Finance:** Price, P/E, fundamentals.
     - **NewsAPI:** Recent financial news.
     - **FRED:** Economic indicators (e.g., GDP).
     - **SEC EDGAR:** Filings and disclosures.
   - Uses caching to avoid redundant API calls.

6. **News Processing & Routing**
   - Each news article is processed through PromptChainingAgent to:
     - Clean and summarize text
     - Classify the event type (e.g., earnings, regulation, product launch)
   - RoutingAgent determines the appropriate specialized model or analysis path.

7. **Evaluator-Optimizer Pipeline**
   - Consolidates all structured data and processed news.
   - Generates a **final investment thesis** with iterative drafting, critique, and refinement.

8. **Evaluation & Metrics**
   - Uses `MultiAgentEvaluator` to:
     - Assess thesis quality (clarity, accuracy, rigor, overall) via LLM evaluation.
     - Measure inter-agent coordination efficiency using conversation logs.
   - Updates `MemoryAgent` with the evaluation results and final thesis summary.

9. **Output**
   - Returns the full `state` dictionary containing:
     - Generated plan
     - Fetched and processed data
     - Final thesis
     - Evaluation metrics

### **Purpose**
`run_analysis` serves as the **end-to-end orchestrator** of the multi-agent financial analysis system.  
It integrates **data retrieval, reasoning, text processing, routing, evaluation, and memory management** to produce a **robust and auditable investment thesis**.


## Financial Analysis Entry Point

In [52]:
if __name__ == "__main__":
    # Prompt user for input with default value
    user_input = input("Enter stock symbol [default: NVDA]: ").strip()

    # Use NVDA if no input is provided
    symbol = user_input.upper() if user_input else "NVDA"

    run_analysis(symbol)

Environment variables loaded from config/aai_520_proj.config
--- Starting Analysis for NVDA ---

--- Retrieved Memory for NVDA ---
{
    "summary": "## Investment Analysis: [AAPL - Apple Inc.]\n\n**Executive Summary:**\n\nThis analysis recommends a **HOLD** rating for Apple Inc. (AAPL). The company exhibits robust financial performance and maintains strong market sentiment. However, its relatively high valuation and identified risks require a more cautious approach. While the mean analyst target price suggests potential upside, the target price variance and market saturation necessitate a neutral stance, awaiting further catalysts for significant growth.\n\n**1. Company Overview:**\n\n*   Apple Inc., headquartered in Cupertino, California, operates in the technology sector, specializing in consumer electronics, software, and online services.\n*   On October 16, 2025, Apple Inc. (formerly \"Usual Stablecoin\") completed a restructuring of divisions to align strategic business units for 

C:\Users\Dell\AppData\Local\Temp\ipykernel_19808\2234080774.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),



--- Generated Plan for NVDA ---
- 1. **Gather Recent News and Press Releases:** Use a news API or financial news aggregator to collect the latest news articles and press releases related to NVDA. This will provide insights into recent developments, product launches, partnerships, and potential market-moving events.
- 2. **Financial Statement Analysis:** Retrieve NVDA's latest financial statements (10-K, 10-Q) from the SEC EDGAR database or a financial data provider. Analyze key financial ratios, revenue growth, profitability metrics, and cash flow trends.  Pay close attention to segments related to data centers, gaming, and automotive.
- 3. **Competitive Landscape Analysis:** Identify NVDA's primary competitors (e.g., AMD, Intel) and assess their strengths, weaknesses, and market positioning. Analyze market share data and pricing strategies to understand NVDA's competitive advantage and potential threats.
- 4. **Market and Industry Trend Analysis:** Analyze the overall market trends f

### Entry Point: Interactive Stock Analysis

This section allows the user to **run the agentic financial analysis workflow interactively**.

---

### **Workflow**

1. **User Input**
   - Prompts the user to enter a stock symbol.
   - Defaults to `"NVDA"` if no input is provided.

2. **Symbol Normalization**
   - Converts the input symbol to uppercase to ensure consistency with financial APIs.

3. **Run Analysis**
   - Calls the `run_analysis(symbol)` function to execute the full **multi-agent financial analysis pipeline**.
   - The function retrieves data, generates a plan, processes news, routes tasks, and produces a final investment thesis with evaluation metrics.